#**2025-2024 Procedures:**


### Configuration:

---



In [ ]:
from google.cloud import bigquery
from datetime import datetime

client = bigquery.Client()
ENV = "dev"             ### REF ENVIRONMENT (DEV/QAS/PRO)  ###
DATASET = "bronze"      ### REF LAYER (BRONZE/SILVER/GOLD) ###
PREFIX = "ltm_kpis_"    ### LTM Process ltm_ / ltm_kpis_   ###
SUFFIX = "25"           ### REF LAYER (BRONZE/SILVER/GOLD) ###
NO_DROP = "TRUE"        ### REF TRUE / FALSE               ###
BUCKET  = "dev_ltm_kpis"

# Company List with original name and new name:
companies = [
    ("MONARCH", "Shape MHS - Monarch"),
    ("CAPITAL", "Shape CHC - Capital"),
    ("FUSION", "Shape TUCSON - Fusion"),
    ("OVER THE MOON", "Shape OTM - Over the Moon - In Phase"),
    ("A1 REFRIGERATION", "Shape AONE - A One"),
    ("LBA", "Shape LBA - LBA"),
    ("PRODIGY PLUMBING", "Shape LBCA - Prodigy Plumbing"),
    ("DEAR", "Shape DEAR - Dear Services"),
    ("HEALTHY HOME", "Shape HHWI - Healthy Home"),
    ("CHAD LOVE", "Shape CLS - Chad Love"),
    ("H AND E COMFORT", "Shape HECS - H&E Comfort"),
    ("JSP", "Shape JSP - John Stevenson"),
    ("ICE COOLING", "Shape ICO - ICE"),
    ("COMPLETE COMFORT", "Shape INDY - Complete Comfort"),
    ("SYNERGY HOME", "Shape LEX - Synergy Home"),
    ("HOWZE", "Shape HZE - Howze Plumbing"),
    ("PROSTAR SERVICES", "Shape NCVA - Prostar"),
    ("ABSOLUTE PLUMBING", "Shape AHS - Absolute Plumbing"),
    ("PERSONAL", "Shape PPP - Personal PHC"),
    ("MY GUY", "Shape MGY - My Guy"),
    ("NORTHSTAR", "Shape NS - Northstar Services"),
    ("SUPREME", "Shape SST - Supreme Service"),
    ("SPARTAN", "Shape JFSP - Spartan Plumbing"),
    ("PERFECT TEMP", "Shape PTHC - Perfect Temp"),
    ("PILOT", "Shape PHS - Pilot Plumbing"),
    ("COSMIC COMFORT", "Shape COS - Jantz Cosmic Comfort"),
    ("GREEN ENERGY", "Shape GEM - Green Energy"),
    ("SHARP", "Shape NEWE - Sharp PHC"),
    ("DAFFY DUCTS", "DAFFY DUCTS")
]

### `Drop Tables: Variable Layer (Bronze/Silver/Gold)`

---



In [ ]:
if (NO_DROP == "FALSE"):
  # Iterar sobre cada compañía en la lista
  for company_name, company_new_name in companies:
      source = f"{PREFIX}{company_name.replace(' ', '_').lower()}"
      table_id = f"gp-technologies-{ENV}.{DATASET}.{source}_{SUFFIX}"

      sql_statements = [
          f"""
          DROP TABLE IF EXISTS {table_id};
          """
      ]

      # Run sql_statements en BigQuery
      for sql in sql_statements:
          query_job = client.query(sql)
          query_job.result()

      # Result:
      print(f"DROP TABLE {table_id} sucessfully.")
else:
      print(f"DROP NOT ACTIVATE.")

DROP NOT ACTIVATE.


### `CLYCLE SCRIPT FOR AUTOSCHEMA, BLANKLINES VALIDATE AND SQL SENTENCES:`
---



In [ ]:
import pandas as pd
import gcsfs

if (NO_DROP == "FALSE"):

  print(datetime.now().strftime("%Y-%m-%d"))

  # Iterar sobre cada compañía en la lista
  for company_name, company_new_name in companies:
      #source   = f"{company_name}"  # Format Path
      source    = f"{company_name.replace(' ', '_')}"  # Format Path
      csv_path  = f"gs://{BUCKET}/{SUFFIX}/{source}.CSV"
      table_id  = source.replace(' ', '_').lower()
      table     = f"gp-technologies-{ENV}.{DATASET}.{PREFIX}{table_id}_{SUFFIX}"

      # Leer CSV desde GCS sin líneas en blanco
      fs = gcsfs.GCSFileSystem()
      with fs.open(csv_path, 'r', encoding="utf-8") as f:
          df = pd.read_csv(f, skip_blank_lines=True)

      # Filtrar filas completamente vacías
      df = df.dropna(how='all')

      # Filtrar filas con solo espacios, tabulaciones o saltos de línea
      df = df[~df.apply(lambda row: row.astype(str).str.strip().eq("").all(), axis=1)]

      # Guardar CSV limpio en GCS
      clean_csv_path = f"gs://{BUCKET}/{SUFFIX}/{source}_clean.csv"
      with fs.open(clean_csv_path, 'w', encoding="utf-8", newline="") as f:
          df.to_csv(f, index=False)

      # Configurar carga en BigQuery
      job_config = bigquery.LoadJobConfig(
          autodetect=True,
          source_format=bigquery.SourceFormat.CSV,
      )

      # Cargar CSV limpio en BigQuery
      load_job = client.load_table_from_uri(clean_csv_path, table, job_config=job_config)
      load_job.result()

      # Ejecutar consultas SQL en BigQuery
      sql_statements = [
          f"ALTER TABLE `{table}` ADD COLUMN IF NOT EXISTS created_at TIMESTAMP;",
          f"ALTER TABLE `{table}` ADD COLUMN IF NOT EXISTS status BOOL;",
          f"""
          UPDATE `{table}`
          SET company_name      = "{company_name}",
              company_new_name  = "{company_new_name}",
              created_by        = "hernann@peachcfo.com",
              tab_title         = "LTM - KPIS",
              created_at        = CURRENT_TIMESTAMP(),
              status            = TRUE
          WHERE status IS NULL OR status = FALSE;
          """,
          f"""
          UPDATE `{table}`
          SET status = FALSE
          WHERE region IS NULL;
          """
      ]

      # Run sql_statements en BigQuery
      for sql in sql_statements:
          query_job = client.query(sql)
          query_job.result()

      # Result:
      destination_table = client.get_table(table)
      print(f"Loaded {destination_table.num_rows} rows for {company_name} ({company_new_name}).")
else:
      print(f"DROP NOT ACTIVATE.")

DROP NOT ACTIVATE.


### `INDIVIDUAL SCRIPT FOR AUTOSCHEMA, BLANKLINES VALIDATE AND SQL SENTENCES:`

In [ ]:
import pandas as pd
import gcsfs

# ("", ""),

company_name = ""
company_new_name = ""

if (company_name != ""):

  #source   = f"{company_name}"  # Format Path
  source    = f"{company_name.replace(' ', '_')}"  # Format Path
  csv_path  = f"gs://dev_ltm_csv/{SUFFIX}/{source}.CSV"
  table_id  = source.replace(' ', '_').lower()
  table     = f"gp-technologies-{ENV}.{DATASET}.{PREFIX}{table_id}_{SUFFIX}"

  # Leer CSV desde GCS sin líneas en blanco
  fs = gcsfs.GCSFileSystem()
  with fs.open(csv_path, 'r', encoding="utf-8") as f:
      df = pd.read_csv(f, skip_blank_lines=True)

  # Filtrar filas completamente vacías
  df = df.dropna(how='all')

  # Filtrar filas con solo espacios, tabulaciones o saltos de línea
  df = df[~df.apply(lambda row: row.astype(str).str.strip().eq("").all(), axis=1)]

  # Guardar CSV limpio en GCS
  clean_csv_path = f"gs://dev_pulse/LTM/{source}_clean.csv"
  with fs.open(clean_csv_path, 'w', encoding="utf-8", newline="") as f:
      df.to_csv(f, index=False)

  # Configurar carga en BigQuery
  job_config = bigquery.LoadJobConfig(
      autodetect=True,
      source_format=bigquery.SourceFormat.CSV,
  )

  # Cargar CSV limpio en BigQuery
  load_job = client.load_table_from_uri(clean_csv_path, table, job_config=job_config)
  load_job.result()

  # Ejecutar consultas SQL en BigQuery
  sql_statements = [
      f"ALTER TABLE `{table}` ADD COLUMN IF NOT EXISTS created_at TIMESTAMP;",
      f"ALTER TABLE `{table}` ADD COLUMN IF NOT EXISTS status BOOL;",
      f"""
      UPDATE `{table}`
      SET company_name      = "{company_name}",
          company_new_name  = "{company_new_name}",
          created_by        = "hernann@peachcfo.com",
          tab_title         = "LTM - KPIS",
          created_at        = CURRENT_TIMESTAMP(),
          status            = TRUE
      WHERE status IS NULL OR status = FALSE;
      """,
      f"""
      UPDATE `{table}`
      SET status = FALSE
      WHERE region IS NULL;
      """
  ]

  # Run sql_statements en BigQuery
  for sql in sql_statements:
      query_job = client.query(sql)
      query_job.result()

  # Result:
  destination_table = client.get_table(table)
  print(f"Loaded {destination_table.num_rows} rows for {company_name} ({company_new_name}).")

else:
      print(f"company_name not exists.")

company_name not exists.
